In [1]:
import os
import pickle
# import pandas as pd
from utils_mocap_viz.generate_views import (    # organize this
    get_output_dir,
    prepare_videos
)
from utils_mocap_viz.animated_phase_analysis import generate_all_animations
from utils_mocap_viz.animated_merged_phase_analysis import animate_merged_phase_analysis
from utils_mocap_viz.kinematic_visualizer import visualize_joint_position
from utils_mocap_viz.video_layout import combine_views, get_video_paths

# from utils_subdivision.gen_distribution_single_plots import find_cycle_phases, kde_estimate

from utils_dance_anim.dance_dot import animate_dance_phase_analysis, save_dance_phase_plot

from utils_pipeline.pipeline_B import *

### Config

In [2]:
file_name = "BKO_E1_D1_02_Maraka"
traj_dir  = "traj_files_presentation"
status    = "included"   # or "excluded"
traj_threshold = "0.001"        # or any other threshold

bvh_dir = os.path.join("data", "bvh_files")
bvh_file = os.path.join(bvh_dir, file_name + "_T")

# path to onsets and cycles csv files
cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
onsets_csv_path = f"data/drum_onsets/{file_name}.csv"

cycle_segs, windows = compute_windows(traj_dir, file_name, status, traj_threshold)

Loaded cycle_segments from BKO_E1_D1_02_Maraka_included_0.001.pkl
cycle_segments: [(23.740333333, 26.174999999666667), (26.174999999666667, 28.663888888666666), (28.663888888666666, 31.07633333333333), (31.07633333333333, 33.48166666633333), (33.48166666633333, 35.851444444), (35.851444444, 38.19277777766667), (38.19277777766667, 40.538555555), (40.538555555, 42.90566666666667), (42.90566666666667, 45.23455555533334), (45.23455555533334, 47.571), (47.571, 49.90033333333333), (49.90033333333333, 52.17588888866667), (52.17588888866667, 54.4496666665), (54.4496666665, 56.688777777666665), (56.688777777666665, 58.955), (58.955, 61.175), (61.175, 63.60966666666667)]
Number of windows for each beat/subdivision:
  beat_1: 17 windows
  beat_2: 17 windows
  beat_3: 17 windows
  beat_4: 17 windows
  subdiv_2: 17 windows
  subdiv_3: 17 windows
  subdiv_5: 17 windows
  subdiv_6: 17 windows
  subdiv_8: 17 windows
  subdiv_9: 17 windows
  subdiv_11: 17 windows
  subdiv_12: 17 windows


### Generate trajectory video + trimmed dance video plots

In [3]:
w_key = "beat_1"
vid_plot_path = f"cycle_videos/{file_name}/{w_key}/"

# traj_tuples = random.sample(windows[w_key], 2)  # Randomly sample 2 tuples from the list

traj_tuples = windows[w_key][:2]

print(windows.keys())
print(traj_tuples)

dict_keys(['beat_1', 'beat_2', 'beat_3', 'beat_4', 'subdiv_2', 'subdiv_3', 'subdiv_5', 'subdiv_6', 'subdiv_8', 'subdiv_9', 'subdiv_11', 'subdiv_12'])
[(21.30566666633333, 26.174999999666667, 23.740333333), (23.68611111066667, 28.663888888666666, 26.174999999666667)]


In [4]:
extract_cycle_videos_and_plots(
    file_name = file_name,
    windows = traj_tuples,  # List of (win_start, win_end, t_poi) tuples
    window_key = w_key,
    base_path_logs = "data/logs_v4_0.007_foot_jun3",            # logs_v4_0.007_foot_jun3       logs_v2_may
    figsize = (10, 3),
    dpi = 200,
    save_dir = "composite_videos",
    )

Windows data:
Window 1:
  Start: 21.306
  End: 26.175
  POI: 23.740
  Duration: 4.869
Window 2:
  Start: 23.686
  End: 28.664
  POI: 26.175
  Duration: 4.978

Foot data ranges:
Left foot time range: 0.733 to 340.450
Right foot time range: 0.267 to 339.721
Number of left foot onsets: 240
Number of right foot onsets: 234

Processing 2 windows
Total frames in trajectory data: 81793
Time range in trajectory data: 0.000 to 340.800

Processing window 1:
  Window time range: 21.306 to 26.175
  Found 3 left foot onsets and 3 right foot onsets
  Left foot onset times: [21.8875     22.47916667 24.95416667]
  Right foot onset times: [21.325      23.72916667 26.17083333]
  Video frames: 1065 to 1308 (50fps)
  Trajectory frames: 5113 to 6281 (240fps)
  Trajectory data points: 1168

Video extraction:
Input video: data/videos/BKO_E1_D1_02_Maraka_pre_R_Mix.mp4
Output video: composite_videos\BKO_E1_D1_02_Maraka\beat_1\videos\BKO_E1_D1_02_Maraka_window_001_21.31_26.17.mp4
Start time: 21.30566666633333
D

### Generate Skeletal video + trimmed_video_mix

###### Available markers: ['Hips', 'LeftHip', 'LeftKnee', 'LeftAnkle', 'LeftToe', 'LeftToeEnd', 'RightHip', 'RightKnee', 'RightAnkle', 'RightToe', 'RightToeEnd', 'Chest', 'Chest2', 'Chest3', 'Chest4', 'LeftCollar', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 'LeftWristEnd', 'RightCollar', 'RightShoulder', 'RightElbow', 'RightWrist', 'RightWristEnd', 'Neck', 'Head', 'HeadEnd']

In [5]:
video_path = os.path.join("data", "videos", f"{file_name}_pre_R_Mix.mp4")
video_size = (1280, 720)

output_dir1 = os.path.join("composite_videos", file_name, w_key, "video_skeleton")
os.makedirs(output_dir1, exist_ok=True)

for start_time, end_time, _ in traj_tuples:
    print(start_time, end_time)
    view_videos = prepare_videos(
        filename= bvh_file,
        start_time= start_time,
        end_time= end_time,
        video_path= video_path,
        video_size= video_size,
        fps= 24,
        output_dir = output_dir1
    )

21.30566666633333 26.174999999666667

Generating front view...

BVH file information:
Frame rate: 240 Hz
Total frames: 81793
Total time: 340.80 seconds
Position columns: ['Time', 'Hips.X', 'Hips.Y', 'Hips.Z', 'LeftHip.X', 'LeftHip.Y', 'LeftHip.Z', 'LeftKnee.X', 'LeftKnee.Y', 'LeftKnee.Z', 'LeftAnkle.X', 'LeftAnkle.Y', 'LeftAnkle.Z', 'LeftToe.X', 'LeftToe.Y', 'LeftToe.Z', 'LeftToeEnd.X', 'LeftToeEnd.Y', 'LeftToeEnd.Z', 'RightHip.X', 'RightHip.Y', 'RightHip.Z', 'RightKnee.X', 'RightKnee.Y', 'RightKnee.Z', 'RightAnkle.X', 'RightAnkle.Y', 'RightAnkle.Z', 'RightToe.X', 'RightToe.Y', 'RightToe.Z', 'RightToeEnd.X', 'RightToeEnd.Y', 'RightToeEnd.Z', 'Chest.X', 'Chest.Y', 'Chest.Z', 'Chest2.X', 'Chest2.Y', 'Chest2.Z', 'Chest3.X', 'Chest3.Y', 'Chest3.Z', 'Chest4.X', 'Chest4.Y', 'Chest4.Z', 'LeftCollar.X', 'LeftCollar.Y', 'LeftCollar.Z', 'LeftShoulder.X', 'LeftShoulder.Y', 'LeftShoulder.Z', 'LeftElbow.X', 'LeftElbow.Y', 'LeftElbow.Z', 'LeftWrist.X', 'LeftWrist.Y', 'LeftWrist.Z', 'LeftWristEnd.X',

### Generate animated kinematic plots: x axis in seconds

In [6]:
# joint_name = "Hips"  
# axis = 'y'      # y is vertical in bvh files

# output_dir2 = os.path.join("composite_videos", file_name, w_key, joint_name)
# os.makedirs(output_dir2, exist_ok=True)


# for start_time, end_time, _ in traj_tuples:
#     print(start_time, end_time)

#     visualize_joint_position(
#         bvh_file=bvh_file + ".bvh",
#         joint_name= joint_name,
#         axis= axis,
#         start_time= start_time,
#         end_time= end_time,
#         output_fps= 24,
#         output_dir= output_dir2,
#         fig_size= (12, 4),  # Half height for joint visualization
#         dpi= 200
#     )

### x axis in beats

In [7]:
def extract_cycle_plots(
    file_name: str,
    windows: list,  # List of (win_start, win_end, t_poi) tuples
    window_key: str,
    joint_name: str,
    axis: str = 'y',
    base_path_logs: str = "data/logs_v2_may",
    frame_rate: float = 240,  # Trajectory data frame rate
    n_beats_per_cycle: int = 4,
    n_subdiv_per_beat: int = 3,
    nn: int = 3,
    output_dir2: str = None,
    figsize: tuple = (10, 3),
    dpi: int = 200
):
    """
    Create trajectory animations for windows around points of interest (beats or subdivisions).
    Each plot shows [-cycle, 0-cycle, +cycle] around the POI.
    """
    # Create save directory if not provided
    if output_dir2 is None:
        output_dir2 = os.path.join("cycle_plots", file_name, window_key, joint_name)
    os.makedirs(output_dir2, exist_ok=True)
    
    # Load joint position data
    dir_csv = "extracted_mocap_csv"
    base_name = os.path.splitext(os.path.basename(file_name))[0]
    worldpos_file = os.path.join(dir_csv, f"{base_name}_T_worldpos.csv")
    
    try:
        world_positions = pd.read_csv(worldpos_file)
        print(f"Successfully loaded CSV with {len(world_positions)} rows")
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        raise
    
    # Get time column and position data
    time_column = world_positions.columns[0]  # First column is time
    times = world_positions[time_column].values
    positions = world_positions[f"{joint_name}.{axis.upper()}"].values
    
    print(f"\nProcessing {len(windows)} windows")
    # print(f"Total frames in trajectory data: {len(times)}")
    # print(f"Time range in trajectory data: {times[0]:.3f} to {times[-1]:.3f}")
    
    # Process each window
    for i, (win_start, win_end, t_poi) in enumerate(windows):
        print(f"\nProcessing window {i+1}:")
        print(f"  Window time range: {win_start:.3f} to {win_end:.3f}")
        
        # Calculate segment times
        start_time = win_start
        end_time = win_end
        duration = end_time - start_time
        downbeat = t_poi  # This is the point of interest (beat or subdivision)
        
        # Calculate avg_cycle from the window duration
        avg_cycle = duration / 2  # Since window is ±1 cycle
        
        # Calculate window parameters
        beat_len = avg_cycle / n_beats_per_cycle
        subdiv_len = beat_len / n_subdiv_per_beat
        half_win = subdiv_len * nn
        
        # Calculate frame numbers for trajectory (240fps)
        traj_start_frame = int(start_time * frame_rate)
        traj_end_frame = int(end_time * frame_rate)
        traj_n_frames = traj_end_frame - traj_start_frame
        
        print(f"  Trajectory frames: {traj_start_frame} to {traj_end_frame} (240fps)")
        
        # Check if we have valid frame numbers
        if traj_start_frame >= traj_end_frame:
            print(f"  Skipping window {i+1}: Invalid frame range (start >= end)")
            continue
        if traj_start_frame < 0:
            print(f"  Skipping window {i+1}: Start frame < 0")
            continue
        if traj_end_frame > len(positions):
            print(f"  Skipping window {i+1}: End frame > total frames")
            continue
        
        # Trim trajectory data using frame numbers at 240fps
        pos_win = positions[traj_start_frame:traj_end_frame]
        t_win = times[traj_start_frame:traj_end_frame]
        
        # Check if we have valid trajectory data
        if len(pos_win) == 0:
            print(f"  Skipping window {i+1}: No trajectory data")
            continue
        
        print(f"  Trajectory data points: {len(pos_win)}")
        
        # Create figure and axis
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        fig.tight_layout(pad=2.0) 
        
        # Calculate all subdivision times for the window
        all_subdiv_times = []
        for beat_idx in range(-n_beats_per_cycle, n_beats_per_cycle + 1):
            beat_time = downbeat + beat_idx * beat_len
            for subdiv_idx in range(n_subdiv_per_beat):
                subdiv_time = beat_time + subdiv_idx * subdiv_len
                if start_time <= subdiv_time <= end_time:
                    all_subdiv_times.append((subdiv_time, beat_idx * n_subdiv_per_beat + subdiv_idx + 1))

        # Plot subdivision lines with appropriate colors
        for subdiv_time, subdiv_num in all_subdiv_times:
            color = get_subdiv_color(subdiv_num)
            # Add yellow glow effect for t=0
            if abs(subdiv_time - downbeat) < 0.001:  # If it's the POI
                ax.axvline(subdiv_time, color='yellow', linestyle='-', linewidth=3, alpha=0.3)
            ax.axvline(subdiv_time, color=color, linestyle='-', linewidth=2, alpha=0.7)
        
        # Plot trajectory
        ax.plot(t_win, pos_win, '-', color='blue', alpha=0.5, label=f'{joint_name} {axis.upper()}')
        
        ax.axvline(downbeat, color='yellow', linewidth=6, alpha=0.3, zorder=0)  # Glow effect
        
        # Set y-axis limits with safety checks
        try:
            y_min = pos_win.min()
            y_max = pos_win.max()
            y_range = y_max - y_min
            ax.set_ylim(y_min - 0.1*y_range, y_max + 0.1*y_range)
        except ValueError as e:
            print(f"  Warning: Could not set y-axis limits: {e}")
            # Set default y-axis limits
            ax.set_ylim(-1, 1)
        
        # Create vertical playhead
        v_playhead, = ax.plot([start_time, start_time], 
                            [y_min - 0.1*y_range, y_max + 0.1*y_range],
                            lw=1, alpha=0.9, color='orange')
        
        # Set up the plot with scaled x-axis
        ax.set_xlabel(f'Beats relative to {window_key}')
        ax.set_ylabel(f'{joint_name} {axis.upper()} Position')
        ax.set_title(f'Window {i+1} | {window_key}: {downbeat:.2f}s')
        ax.grid(True, alpha=0.3)
        
        # Scale x-axis to show beats instead of cycles
        x_ticks = np.arange(-n_beats_per_cycle, n_beats_per_cycle + 1)
        x_tick_positions = downbeat + x_ticks * beat_len
        ax.set_xticks(x_tick_positions)
        ax.set_xticklabels(x_ticks)
        
        # Add legend
        custom = [
            Line2D([0],[0], color='blue', lw=1),
            Line2D([0],[0], color='black', lw=1),
            Line2D([0],[0], color='green', lw=1),
            Line2D([0],[0], color='red', lw=1),
        ]
        labels = [
            f"{joint_name} {axis.upper()}", 
            "Subdiv-1 (1,4,7,10)", 
            "Subdiv-2 (2,5,8,11)", 
            "Subdiv-3 (3,6,9,12)"
        ]
        ax.legend(custom, labels, loc='upper left', framealpha=0.3, fontsize=6)
        
        def update(frame):
            v_playhead.set_xdata([frame, frame])
            ax.set_title(f'Cycle {i+1} | {window_key}: {downbeat:.2f}s | Time: {frame:.2f}s')
            return v_playhead,
        
        # Create animation frames at 24fps
        n_frames = int(duration * 24)
        frames = np.linspace(start_time, end_time, n_frames)
        anim = animation.FuncAnimation(
            fig, update, frames=frames,
            interval=1000/24,  # 24fps
            blit=True
        )
        
        # Save animation
        plot_output_path = os.path.join(output_dir2, f"{file_name}_window_{i+1:03d}_{start_time:.2f}_{end_time:.2f}.mp4")
        writer = animation.FFMpegWriter(fps= 24, 
                                        bitrate=2000,
                                        codec='libx264',  # Specify codec
                                        # extra_args=['-preset', 'ultrafast']
                                        )  # 24fps
        anim.save(plot_output_path, writer=writer)
        plt.close(fig)
        
        print(f"  Plot saved: {plot_output_path}")
        print(f"  Plot duration: {len(frames)/24:.3f}s")
    
    print("\nProcessing complete!")

In [8]:
joint_name = "Hips"  
axis = 'y'      # y is vertical in bvh files

output_dir2 = os.path.join("composite_videos", file_name, w_key, joint_name)
os.makedirs(output_dir2, exist_ok=True)

extract_cycle_plots(
file_name= file_name,
windows= traj_tuples,
window_key= w_key,
joint_name= joint_name,
axis="y",
output_dir2= output_dir2,
figsize = (10, 3),  # 2000 x 600 px
dpi= 200,
)

Successfully loaded CSV with 81793 rows

Processing 2 windows

Processing window 1:
  Window time range: 21.306 to 26.175
  Trajectory frames: 5113 to 6281 (240fps)
  Trajectory data points: 1168
  Plot saved: composite_videos\BKO_E1_D1_02_Maraka\beat_1\Hips\BKO_E1_D1_02_Maraka_window_001_21.31_26.17.mp4
  Plot duration: 4.833s

Processing window 2:
  Window time range: 23.686 to 28.664
  Trajectory frames: 5684 to 6879 (240fps)
  Trajectory data points: 1195
  Plot saved: composite_videos\BKO_E1_D1_02_Maraka\beat_1\Hips\BKO_E1_D1_02_Maraka_window_002_23.69_28.66.mp4
  Plot duration: 4.958s

Processing complete!


### Prepare for concatenation

In [9]:
def extract_category(filename):
    """
    Given a filename like "front_view_56.7_61.2.mp4" or
    "BKO_E1_D1_02_Maraka_pre_R_Mix_trimmed_56.7_61.2.mp4",
    return the category portion before the last two underscore-separated tokens.
    """
    name, _ = os.path.splitext(filename)     # strip .mp4
    parts = name.split('_')
    # Last two parts are start and end times, so category is everything before them
    if len(parts) > 2:
        return "_".join(parts[:-2])
    return name   # fallback if unexpected format

def write_all_categories(files, output_dir, video_dir):
    """
    From a list of filenames, group by category (as defined by extract_category),
    and write each group into its own .txt file in output_dir.
    """
    # os.makedirs(output_dir, exist_ok=True)

    # Group filenames by category
    categories = {}
    for fname in files:
        cat = extract_category(fname)
        categories.setdefault(cat, []).append(fname)

    # Write each category's filenames to a separate text file
    for cat, fnames in categories.items():
        txt_path = os.path.join(output_dir, f"{cat}.txt")
        with open(txt_path, "w") as fw:
            for f in fnames:
                if video_dir:
                    rel_path = os.path.relpath(os.path.join(video_dir, f), os.path.dirname(txt_path))
                    fw.write(f"file '{rel_path}'\n")
                else:
                    fw.write(f + "\n")  
                

def create_concat_file(video_dir, output_file, prefix):
    """Create a text file listing all videos in order for concatenation"""
    with open(output_file, 'w') as f:
        # Get all video files and sort them
        video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
        # Write each file path - use relative path from the text file location
        for video in video_files:
            # Get relative path from output_file to video_dir
            rel_path = os.path.relpath(os.path.join(video_dir, video), os.path.dirname(output_file))
            f.write(f"file '{rel_path}'\n") 
            
def concatenate_and_overlay_videos(file_name, joint_name,  save_dir):
    """Concatenate cycle videos and plot videos, then overlay them"""
    video_dir = os.path.join(save_dir, "videos")
    plot_dir = os.path.join(save_dir, "plots")
    joint_dir = os.path.join(save_dir, joint_name)
    vid_skel_dir = os.path.join(save_dir, "video_skeleton")

    # Create text files for concatenation
    video_list = os.path.join(save_dir, "video_list.txt")
    plot_list = os.path.join(save_dir, "plot_list.txt")
    joint_list = os.path.join(save_dir, "joint_list.txt")
    
    front_view_list = os.path.join(save_dir, "front_view.txt")
    left_view_list = os.path.join(save_dir, "left_view.txt")
    right_view_list = os.path.join(save_dir, "right_view.txt")
    top_view_list = os.path.join(save_dir, "top_view.txt")
    
    
    mp4_file_list = [f for f in os.listdir(vid_skel_dir) if f.lower().endswith(".mp4")]
    write_all_categories(mp4_file_list, save_dir, video_dir = vid_skel_dir)
    
    
    # Check if directories exist
    if not os.path.exists(video_dir):
        print(f"Video directory not found: {video_dir}")
        return
    if not os.path.exists(plot_dir):
        print(f"Plot directory not found: {plot_dir}")
        return
        
    # Check if text files already exist
    if os.path.exists(video_list) and os.path.exists(plot_list):
        print("Concatenation files already exist, skipping creation")
    else:
        print("Creating concatenation files...")
        create_concat_file(video_dir, video_list, f"{file_name}_cycle_")
        create_concat_file(plot_dir, plot_list, f"{file_name}_cycle_")
        create_concat_file(joint_dir, joint_list, f"{file_name}_cycle_")
    
    
    def concatenate_videos(video_list, save_dir, save_name):
        
        concat_video = os.path.join(save_dir, f"{save_name}.mp4")
        try:
            result = subprocess.run([
                'ffmpeg', '-y',
                '-f', 'concat',
                '-safe', '0',
                '-i', video_list,
                '-c', 'copy',
                concat_video
            ], capture_output=True, text=True)
            if result.returncode != 0:
                print("Error concatenating videos:", result.stderr)
                return
        except Exception as e:
            print("Error running ffmpeg:", str(e))
            return
    
    concatenate_videos(video_list, save_dir, f"video_mix_concat")
    concatenate_videos(plot_list, save_dir, f"plot_concat")
    concatenate_videos(joint_list, save_dir, f"joint_{joint_name}_concat")
    
    concatenate_videos(front_view_list, save_dir, f"front_view_concat")
    concatenate_videos(left_view_list, save_dir, f"left_view_concat")
    concatenate_videos(right_view_list, save_dir, f"right_view_concat")
    concatenate_videos(top_view_list, save_dir, f"top_view_concat") 

    
    # print(f"Concatenated plot saved: {concat_plot}")
    # print(f"Concatenated video saved: {concat_video}\n")

In [10]:
joint_name = "Hips"

vid_plot_path = os.path.join("composite_videos", file_name, w_key)

concatenate_and_overlay_videos(file_name, joint_name, vid_plot_path)        # modify

Creating concatenation files...


### Generate Composite Video

In [11]:
# file_name = "BKO_E1_D1_02_Maraka"
# w_key = "beat_1"
# vid_plot_path = os.path.join("composite_videos", file_name, w_key)

concat_file_list = [f for f in os.listdir(vid_plot_path) if f.lower().endswith(".mp4")]
concat_dict = {
    f.replace('_concat.mp4', ''): os.path.join(vid_plot_path, f) 
    for f in concat_file_list
}

concat_dict

{'front_view': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\front_view_concat.mp4',
 'joint_Hips': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\joint_Hips_concat.mp4',
 'left_view': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\left_view_concat.mp4',
 'plot': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\plot_concat.mp4',
 'right_view': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\right_view_concat.mp4',
 'top_view': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\top_view_concat.mp4',
 'video_mix': 'composite_videos\\BKO_E1_D1_02_Maraka\\beat_1\\video_mix_concat.mp4'}

### test

In [12]:
import os
import subprocess

def resize_video(video_path, width, height, save_dir):
    """
    Resize a video to the specified width and height using ffmpeg,
    with debug‐level output.

    Parameters:
    - video_path: str, path to the input video file
    - width: int, target width in pixels
    - height: int, target height in pixels
    - save_dir: str, directory where the resized video will be saved

    The output filename will be: <original_basename>_<width>x<height>.mp4
    """
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Derive output filename from input basename
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    output_filename = f"{base_name}.mp4"   # f"{base_name}_{width}x{height}.mp4"
    output_path = os.path.join(save_dir, output_filename)

    # Build ffmpeg command with debug-level logging
    cmd = [
        "ffmpeg",
        "-y",                   # overwrite output if it exists
        "-loglevel", "debug",   # show full debug output
        "-i", video_path,
        "-vf", f"scale={width}:{height}",
        "-c:v", "libx264",
        "-crf", "18",
        "-preset", "slow",
        output_path
    ]

    # Run ffmpeg and capture stdout/stderr
    result = subprocess.run(cmd, capture_output=True, text=True)

    # Print debug output
    # print("=== ffmpeg stdout ===")
    # print(result.stdout)
    # print("=== ffmpeg stderr ===")
    # print(result.stderr)

    # Check return code and report
    if result.returncode == 0:
        print(f"Resizing succeeded, output saved to: {output_path}")
    else:
        print(f"ffmpeg failed with return code {result.returncode}")

    return output_path if result.returncode == 0 else None

In [13]:
view_videos = {
    'video_mix': concat_dict['video_mix'],  
    'plot': concat_dict['plot'],    
    'front_view': concat_dict['front_view'],
    'joint_Hips': concat_dict['joint_Hips'],
    'left_view': concat_dict['left_view'],
    'right_view': concat_dict['right_view'],
    'top_view': concat_dict['top_view'],
}

canvas_w, canvas_h = 1920, 1080 
composite_layout = [
    # Top row - side by side
    {'view': 'video_mix', 'x': 0, 'y': 0, 'width': 960, 'height': 540},
    {'view': 'front_view', 'x': 960, 'y': 0, 'width': 960, 'height': 540},
    
    # Bottom row - stacked vertically
    {'view': 'joint_Hips', 'x': 0, 'y': 540, 'width': 1920, 'height': 270},
    {'view': 'plot', 'x': 0, 'y': 810, 'width': 1920, 'height': 270},
]

saved_resized_dir = os.path.join(vid_plot_path, "temp_resized")
os.makedirs(saved_resized_dir, exist_ok=True)

composite_video_elements = []

for video_element in composite_layout:
    video_path = view_videos[video_element['view']]
    v_width, v_height = video_element['width'], video_element['height']
    x_pos_pxl, y_pos_pxl = video_element['x'], video_element['y']
    
    resized_path = resize_video(video_path, v_width, v_height, saved_resized_dir)
    
    composite_video_elements.append({"view": video_element['view'], 
                           "vid_path": resized_path,
                           "x_pos_pxl": x_pos_pxl,
                           "y_pos_pxl": y_pos_pxl,
                           })







Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_02_Maraka\beat_1\temp_resized\video_mix_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_02_Maraka\beat_1\temp_resized\front_view_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_02_Maraka\beat_1\temp_resized\joint_Hips_concat.mp4
Resizing succeeded, output saved to: composite_videos\BKO_E1_D1_02_Maraka\beat_1\temp_resized\plot_concat.mp4


In [14]:
# Create a list of dictionaries with just the paths and positions
video_positions = []
for element in composite_video_elements:
    video_positions.append({
        'path': element['vid_path'],
        'x': element['x_pos_pxl'],
        'y': element['y_pos_pxl']
    })

# Build the ffmpeg command
ffmpeg_inputs = []
for pos in video_positions:
    ffmpeg_inputs.extend(['-i', pos['path']])

# Create the xstack layout string
# Format: xstack=inputs=4:layout=0_0|w0_0|0_h0|w0_h0
layout = []
for pos in video_positions:
    layout.append(f"{pos['x']}_{pos['y']}")

xstack_layout = "|".join(layout)

final_out = os.path.join(vid_plot_path, "final_output.mp4")
ffmpeg_cmd = [
    'ffmpeg', '-y',
    *ffmpeg_inputs,
    '-filter_complex', f'xstack=inputs={len(video_positions)}:layout={xstack_layout}[v]',
    '-map', '[v]',
    '-map', '0:a?', '-c:a', 'aac', '-b:a', '192k',
    '-c:v', 'libx264',   #'libx264',
    '-crf', '23',
    '-preset', 'ultrafast',
    final_out
]

# Execute the command
import subprocess
try:
    subprocess.run(ffmpeg_cmd, check=True)
    print("Video successfully created as final_output.mp4")
except subprocess.CalledProcessError as e:
    print(f"Error creating video: {e}")

Video successfully created as final_output.mp4
